In [19]:
with open('myfile.txt', 'rb') as f:
    binary_data = f.read()
    binary_string = ''.join(format(byte, '08b') for byte in binary_data)


In [45]:
binary_string

'010010010110111000100000011101000110100001100101001000000110110101100101011011010110111100100000011001100111001001101111011011010010000001110100011010000110010100100000011000010110010001110110011001010111001001110100011010010111001101101001011011100110011100100000011001000110100101110010011001010110001101110100011011110111001000100000011011110110011000100000011101000110100001100101001000000101001101110101011100000110010101110010001000000101001101100011011100100110010101100101011011100010000001001101011011110111011001101001011001010010000001010000011100100110111101100100011101010110001101110100011010010110111101101110001000000100001101101111011011010111000001100001011011100111100100101100001000000110100101110100001000000110100101110011001000000111001101110100011000010111010001100101011001000010000001110100011010000110000101110100001000000111001001100101011001000111010101100011011001010110010000100000011100000110010101101111011100000110110001100101001000000110000101110100011101000110010

In [39]:
from PIL import Image

# Open the image
# image = Image.open('example.jpg')
new_image = Image.new('RGB', (640, 480), (0, 0, 0))

# Get the size of the image
width, height = new_image.size

# Loop over each pixel in the image and set its value
for x in range(width):
    for y in range(height):
        # Set the color of the pixel
        new_image.putpixel((x, y), (256, 256, 256))  # Red color

# Save the new image
new_image.save('./images/example_pixels_0002.jpg')


In [ ]:
def create_image(index):
    pass

In [ ]:
total_length=len(binary_string)
index=1
current_index=0
while(total_length>307200):
    create_image(index,current_index)


In [42]:
import imageio



# Create a writer object to write the MP4 video file
writer = imageio.get_writer('./videos/example.mp4', fps=1)

# Write the image to the video file for 10 seconds
for i in range(2):
# Load the JPG image file
    image = imageio.imread('./images/example_pixels_000'+str(i+1)+'.jpg')
    writer.append_data(image)

# Close the writer object
writer.close()


/tmp/ipykernel_4221/3666013436.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread('./images/example_pixels_000'+str(i+1)+'.jpg')
